In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Preparation & Preprocessing

The target feature ( machine failure ) is in unbalance state and the goal of this work is not find solutions for this issue but only to explore and study the SSL & AL techniques , that's why we'll extract the dataset to a balanced version which we'll use on this following work.

In [126]:
data_labeled = pd.read_csv("data/labeled.csv")
data_unlabeled = pd.read_csv("data/unlabeled.csv")
columns = data_labeled.columns
data_labeled

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,M22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,136425,H38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,136426,L54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0
136427,136427,L53876,L,301.7,310.9,1447,46.3,42,0,0,0,0,0,0


In [127]:
print("labeledset")
print("Machine failure (from): ",np.unique(data_labeled.loc[:,"Machine failure"],return_counts=True)[1])

# number of rows to keep in majority class
count_to_extract = np.unique(data_labeled.loc[:,"Machine failure"],return_counts=True)[1][1]

# extracting from the majority class
new_data_labeled_0 = np.array(data_labeled[data_labeled.loc[:,"Machine failure"] == 0])
np.random.shuffle(new_data_labeled_0)
new_data_labeled_0 = new_data_labeled_0[0:count_to_extract]

# concatenation to get the ne dataset
new_data_labeled_1 = np.array(data_labeled[data_labeled.loc[:,"Machine failure"] == 1])
new_data_labeled = np.vstack((new_data_labeled_1,new_data_labeled_0))
data_labeled = pd.DataFrame(new_data_labeled,columns=columns)

print("Machine failure (to): ",np.unique(data_labeled.loc[:,"Machine failure"],return_counts=True)[1])

labeledset
Machine failure (from):  [134281   2148]
Machine failure (to):  [2148 2148]


In [128]:
# drop ID's columns
data_labeled = data_labeled.drop(['Product ID','id'],axis=1)
data_unlabeled = data_unlabeled.drop(['Product ID','id'],axis=1)

# one hot to categorial features
data_labeled = pd.get_dummies(data_labeled,columns=['Type'])
data_unlabeled = pd.get_dummies(data_unlabeled,columns=['Type'])
data_labeled

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,303.9,312.8,1345,56.5,21,1,0,0,0,0,0,0,1,0
1,302.5,310.4,1307,54.8,174,1,0,1,0,0,0,0,1,0
2,300.7,309.7,1878,27.9,20,1,0,0,0,0,0,0,0,1
3,297.3,308.6,1258,61.8,144,1,0,0,0,1,0,0,1,0
4,303.0,311.3,1341,51.0,174,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,298.1,308.8,1459,31.6,145,0,0,0,0,0,0,0,1,0
4292,300.8,310.5,1447,39.8,114,0,0,0,0,0,0,0,1,0
4293,301.7,310.6,1636,36.6,191,0,0,0,0,0,0,1,0,0
4294,299.4,309.1,1487,38.4,55,0,0,0,0,0,0,0,1,0


In [137]:
# normalization min-max to a column
def normalize_min_max(column):
    return (column - column.min()) / (column.max() - column.min())

# to all dataset
data_labeled = data_labeled.apply(normalize_min_max)
data_unlabeled = data_unlabeled.apply(normalize_min_max)
data = pd.concat((data_labeled,data_unlabeled))
data

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,0.943182,0.894737,0.096188,0.723901,0.083004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.784091,0.578947,0.0739,0.700549,0.687747,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.579545,0.486842,0.408798,0.331044,0.079051,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.193182,0.342105,0.045161,0.796703,0.56917,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.840909,0.697368,0.093842,0.648352,0.687747,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,0.769231,0.703704,0.183935,0.502747,0.059289,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
90950,0.285714,0.506173,0.217695,0.412088,0.12253,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
90951,0.032967,0.061728,0.19383,0.516484,0.73913,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
90952,0.307692,0.259259,0.213038,0.501374,0.272727,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [147]:
# extracting inputs data and target feature
inputs = data.drop('Machine failure',axis=1)
target = data.loc[:,'Machine failure']

After this step of preprocessing we get a complete dataset with a target column labeled at 4.5% (4296 rows with labels and 90954 without)